In [1]:
import numpy as np
import pandas as pd
from os.path import join

from pylab import rcParams
import matplotlib.pyplot as plt
%matplotlib qt

import nilmtk
from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.disaggregate import CombinatorialOptimisation, fhmm_exact
from nilmtk.utils import print_dict
from nilmtk.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 25 days


In [2]:
import sys

In [3]:
sys.path.append("../../code/fridge/")

In [4]:
ds = DataSet("/Users/nipunbatra/wikienergy-2013.h5")


In [5]:
ds.buildings

OrderedDict([(1, Building(instance=1, dataset='Dataport')), (10, Building(instance=10, dataset='Dataport')), (11, Building(instance=11, dataset='Dataport')), (12, Building(instance=12, dataset='Dataport')), (13, Building(instance=13, dataset='Dataport')), (14, Building(instance=14, dataset='Dataport')), (15, Building(instance=15, dataset='Dataport')), (16, Building(instance=16, dataset='Dataport')), (17, Building(instance=17, dataset='Dataport')), (18, Building(instance=18, dataset='Dataport')), (19, Building(instance=19, dataset='Dataport')), (2, Building(instance=2, dataset='Dataport')), (20, Building(instance=20, dataset='Dataport')), (21, Building(instance=21, dataset='Dataport')), (22, Building(instance=22, dataset='Dataport')), (23, Building(instance=23, dataset='Dataport')), (24, Building(instance=24, dataset='Dataport')), (25, Building(instance=25, dataset='Dataport')), (26, Building(instance=26, dataset='Dataport')), (27, Building(instance=27, dataset='Dataport')), (28, Buildi

In [16]:
building_number = 2


In [17]:
elec = ds.buildings[building_number].elec
mains = elec.mains()
elec.appliances



[Appliance(type='spin dryer', instance=1),
 Appliance(type='air conditioner', instance=1),
 Appliance(type='stove', instance=1)]

In [18]:
from nilmtk.disaggregate.hart_85 import Hart85
h = Hart85()
h.train(elec.mains())

Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.


In [19]:
h.centroids

,"(power, active)"
0,227.799286
1,3326.892097


In [20]:
df_ac = elec['air conditioner'].load().next()[('power','active')]

In [23]:
df_ac[df_ac>100].describe()

count    14604.000000
mean      2734.961426
std        443.669983
min        102.000000
25%       2588.000000
50%       2803.500000
75%       3022.250000
max       3345.000000
Name: (power, active), dtype: float64

In [46]:
split_point = elec.train_test_split(train_fraction=0.2)

In [47]:
split_point.date()

datetime.date(2014, 4, 13)

In [48]:
train = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")
test = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")

In [49]:
train.set_window(end=split_point.date())



In [11]:
train.buildings[11].elec.plot()

In [16]:
test.set_window(start=split_point.date())

In [17]:
co = CombinatorialOptimisation()
co.train(elec)

Training model for submeter 'ElecMeter(instance=2, building=11, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])'
Training model for submeter 'ElecMeter(instance=3, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=1)])'
Training model for submeter 'ElecMeter(instance=4, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=2)])'
Training model for submeter 'ElecMeter(instance=5, building=11, dataset='WikiEnergy', appliances=[Appliance(type='dish washer', instance=1)])'
Training model for submeter 'ElecMeter(instance=6, building=11, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])'
Training model for submeter 'ElecMeter(instance=7, building=11, dataset='WikiEnergy', appliances=[Appliance(type='electric furnace', instance=1)])'
Training model for submeter 'ElecMeter(instance=8, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=3)])'
Tra

In [13]:
disag_filename = 'wikienergy-disag-co-new.h5'
output = HDFDataStore(disag_filename, 'w')
co.disaggregate(elec.mains(), output)
output.close()

vampire_power = 0.0 watts
Estimating power demand for 'ElecMeter(instance=2, building=11, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])'
Estimating power demand for 'ElecMeter(instance=3, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=1)])'
Estimating power demand for 'ElecMeter(instance=4, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=2)])'
Estimating power demand for 'ElecMeter(instance=5, building=11, dataset='WikiEnergy', appliances=[Appliance(type='dish washer', instance=1)])'
Estimating power demand for 'ElecMeter(instance=6, building=11, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])'
Estimating power demand for 'ElecMeter(instance=7, building=11, dataset='WikiEnergy', appliances=[Appliance(type='electric furnace', instance=1)])'
Estimating power demand for 'ElecMeter(instance=8, building=11, dataset='WikiEnergy', appliances=[Appliance(type='so

In [16]:
disag = DataSet(disag_filename)
disag_elec_co = disag.buildings[building_number].elec

In [17]:
pred_df_co = disag_elec['fridge'].load().next()[('power','active')]

In [18]:
fhmm = fhmm_exact.FHMM()
fhmm.train(elec)

Training model for submeter 'ElecMeter(instance=2, building=11, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])'
Training model for submeter 'ElecMeter(instance=3, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=1)])'
Training model for submeter 'ElecMeter(instance=4, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=2)])'
Training model for submeter 'ElecMeter(instance=5, building=11, dataset='WikiEnergy', appliances=[Appliance(type='dish washer', instance=1)])'
Training model for submeter 'ElecMeter(instance=6, building=11, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])'
Training model for submeter 'ElecMeter(instance=7, building=11, dataset='WikiEnergy', appliances=[Appliance(type='electric furnace', instance=1)])'
Training model for submeter 'ElecMeter(instance=8, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=3)])'
Tra

In [19]:
disag_filename = 'wikienergy-disag-fhmm.h5'
output = HDFDataStore(disag_filename, 'w')
fhmm.disaggregate(elec.mains(), output)

KeyboardInterrupt: 

In [ ]:
disag = DataSet(disag_filename)
disag_elec = disag.buildings[building_number].elec 

In [ ]:
pred_df_fhmm = disag_elec['fridge'].load().next()[('power','active')]

In [68]:
h.centroids

,"(power, active)"
0,127.233546
1,2535.203256
2,3360.298845
3,1166.317494
4,889.568344
5,1535.951389
6,3823.108170
7,1904.520833
8,4933.812500


In [61]:
h.transients[h.transients['active transition'].abs()>1000].head(20)

,active transition
2014-04-01 07:04:00-05:00,3638.857143
2014-04-01 07:21:00-05:00,-3974.500000
2014-04-01 11:33:00-05:00,3246.475000
2014-04-01 12:01:00-05:00,-3241.200000
2014-04-01 18:12:00-05:00,3247.000000
2014-04-01 18:19:00-05:00,-3862.916667
2014-04-01 19:39:00-05:00,4229.404762
2014-04-01 20:55:00-05:00,-1276.000000
2014-04-01 21:50:00-05:00,-2596.033333
2014-04-02 07:11:00-05:00,1675.400000


In [63]:
#ax=a.plot()
h.pair_df[h.pair_df["T1 Active"]>0].head()

,T1 Time,T1 Active,T2 Time,T2 Active
0,2014-04-01 05:53:00,92.616071,2014-04-01 06:10:00,-109.758929
1,2014-04-01 05:45:00,121.362126,2014-04-01 06:18:00,-103.317460
2,2014-04-01 07:18:00,107.666667,2014-04-01 07:22:00,-116.000000
3,2014-04-01 07:05:00,114.702703,2014-04-01 07:35:00,-208.432624
4,2014-04-01 08:44:00,98.330144,2014-04-01 08:56:00,-120.659091


In [66]:
a = train.buildings[building_number].elec[('air conditioner')].load().next()[('power','active')]
b = train.buildings[building_number].elec.mains().load().next()[('power','active')]

In [67]:
ax = a.head(30000).plot()
b.head(30000).plot(ax=ax)



In [24]:
disag_filename = 'wikienergy-disag-hart.h5'
output = HDFDataStore(disag_filename, 'w')
h.disaggregate(elec.mains(), output)
disag = DataSet(disag_filename)
disag_elec = disag.buildings[building_number].elec

Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.


In [27]:
ax1 = disag_elec['unknown', 0].load().next().plot()
pred_df_co.plot(ax=ax1, label="Combinatorial Optimisation")
elec['fridge', 1].load().next().plot(ax=ax1)
ax1.legend(["Predicted Hart", "Predicted CO","Ground truth"]);
plt.ylabel("Power (W)")
plt.xlabel("Time");